In [4]:
import warnings
warnings.filterwarnings('ignore')
%run process #put original file 'ncaa' in same directory
%run processgoon
import gpfeatures
%run gpfeatures
data=gpfeatures.import_()
X_train=data[0]
Y_train=data[1]
X_test=data[2]

D:\March_madness\gpfeatures.py:71: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  X_test.loc[:] = stdSc.transform(X_test.loc[:])
D:\March_madness\gpfeatures.py:71: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  X_test.loc[:] = stdSc.transform(X_test.loc[:])


In [2]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from boruta import BorutaPy

# load X and y
# NOTE BorutaPy accepts numpy arrays only, hence the .values attribute
X =X_train
y =Y_train
y = y.ravel()

# define random forest classifier, with utilising all cores and
# sampling in proportion to y labels
rf = ExtraTreesClassifier(n_jobs=-1, max_depth=8,n_estimators=247)

# define Boruta feature selection method
feat_selector = BorutaPy(rf, n_estimators='auto', verbose=2, max_iter=300,random_state=27)

# find all relevant features - 5 features should be selected
feat_selector.fit(X.values, y)

# check selected features - first 5 features are selected
feat_selector.support_

# check ranking of features
feat_selector.ranking_

# call transform() on X to filter it down to selected features
train_filtered = feat_selector.transform(X.values)
test_filtered=feat_selector.transform(X_test.values)
train_filtered=pd.DataFrame(train_filtered)
test_filtered=pd.DataFrame(test_filtered)

Iteration: 	1 / 300
Confirmed: 	0
Tentative: 	65
Rejected: 	0
Iteration: 	2 / 300
Confirmed: 	0
Tentative: 	65
Rejected: 	0
Iteration: 	3 / 300
Confirmed: 	0
Tentative: 	65
Rejected: 	0
Iteration: 	4 / 300
Confirmed: 	0
Tentative: 	65
Rejected: 	0
Iteration: 	5 / 300
Confirmed: 	0
Tentative: 	65
Rejected: 	0
Iteration: 	6 / 300
Confirmed: 	0
Tentative: 	65
Rejected: 	0
Iteration: 	7 / 300
Confirmed: 	0
Tentative: 	65
Rejected: 	0
Iteration: 	8 / 300
Confirmed: 	26
Tentative: 	17
Rejected: 	22
Iteration: 	9 / 300
Confirmed: 	26
Tentative: 	17
Rejected: 	22
Iteration: 	10 / 300
Confirmed: 	26
Tentative: 	17
Rejected: 	22
Iteration: 	11 / 300
Confirmed: 	26
Tentative: 	17
Rejected: 	22
Iteration: 	12 / 300
Confirmed: 	28
Tentative: 	15
Rejected: 	22
Iteration: 	13 / 300
Confirmed: 	28
Tentative: 	15
Rejected: 	22
Iteration: 	14 / 300
Confirmed: 	28
Tentative: 	15
Rejected: 	22
Iteration: 	15 / 300
Confirmed: 	28
Tentative: 	15
Rejected: 	22
Iteration: 	16 / 300
Confirmed: 	28
Tentative: 	

Rejected: 	29
Iteration: 	129 / 300
Confirmed: 	29
Tentative: 	7
Rejected: 	29
Iteration: 	130 / 300
Confirmed: 	29
Tentative: 	7
Rejected: 	29
Iteration: 	131 / 300
Confirmed: 	29
Tentative: 	7
Rejected: 	29
Iteration: 	132 / 300
Confirmed: 	29
Tentative: 	7
Rejected: 	29
Iteration: 	133 / 300
Confirmed: 	29
Tentative: 	7
Rejected: 	29
Iteration: 	134 / 300
Confirmed: 	29
Tentative: 	7
Rejected: 	29
Iteration: 	135 / 300
Confirmed: 	29
Tentative: 	7
Rejected: 	29
Iteration: 	136 / 300
Confirmed: 	29
Tentative: 	7
Rejected: 	29
Iteration: 	137 / 300
Confirmed: 	29
Tentative: 	7
Rejected: 	29
Iteration: 	138 / 300
Confirmed: 	29
Tentative: 	7
Rejected: 	29
Iteration: 	139 / 300
Confirmed: 	29
Tentative: 	7
Rejected: 	29
Iteration: 	140 / 300
Confirmed: 	29
Tentative: 	7
Rejected: 	29
Iteration: 	141 / 300
Confirmed: 	29
Tentative: 	7
Rejected: 	29
Iteration: 	142 / 300
Confirmed: 	29
Tentative: 	7
Rejected: 	29
Iteration: 	143 / 300
Confirmed: 	29
Tentative: 	7
Rejected: 	29
Iteration: 

In [4]:
print(X_train.shape,X_test.shape)
gp_train,gp_test=gpfeatures.gp(train_filtered,test_filtered)
print(gp_train.shape,gp_test.shape)
lda_train,lda_test=gpfeatures.lda(gp_train,gp_test)
print(lda_train.shape,lda_test.shape)

(1112, 65) (2278, 65)
(1112, 36) (2278, 36)


D:\anaconda\anacondaApp\lib\site-packages\sklearn\discriminant_analysis.py:388: UserWarning: Variables are collinear.
  warnings.warn("Variables are collinear.")


(1112, 37) (2278, 37)


In [2]:
#a=X_train.drop(['oppftpct','ftpct','slot','f3grate','oppf3grate','tempo','adjtempo','opp_stlrate','opp_arate'],axis=1)

In [5]:
%store X_train
%store X_test
%store Y_train
%store gp_train
%store lda_train
%store lda_test

Stored 'X_train' (DataFrame)
Stored 'X_test' (DataFrame)
Stored 'Y_train' (Series)
Stored 'gp_train' (DataFrame)
Stored 'lda_train' (DataFrame)
Stored 'lda_test' (DataFrame)


In [6]:

def tpot(X):
    global train_score,test_score
    x_train, x_test, y_train, y_test = train_test_split(X,
        Y_train, train_size=0.8, test_size=0.2)
    tpot=TPOTClassifier(generations=10,population_size=20,scoring='neg_log_loss',verbosity=2,warm_start=True)
    tpot.fit(x_train, y_train)
    train_score=tpot.score(x_train,y_train)
    test_score=tpot.score(x_test, y_test)
    tpot.export('tpot_pipeline.py')
    print(train_score)
    print(test_score)
    return tpot
    

In [7]:
tpot=tpot(lda_train)

Generation 1 - Current best internal CV score: -0.5119039233631186
Generation 2 - Current best internal CV score: -0.5119039233631186
Generation 3 - Current best internal CV score: -0.5094472678209556
Generation 4 - Current best internal CV score: -0.507158543101581
Generation 5 - Current best internal CV score: -0.507158543101581
Generation 6 - Current best internal CV score: -0.5056773331850211
Generation 7 - Current best internal CV score: -0.5056773331850211
Generation 8 - Current best internal CV score: -0.5056769683953861
Generation 9 - Current best internal CV score: -0.49918839996163583
Generation 10 - Current best internal CV score: -0.49918839996163583

Best pipeline: LogisticRegression(SelectPercentile(RFE(input_matrix, criterion=entropy, max_features=0.05, n_estimators=100, step=0.15), percentile=17), C=0.1, dual=True, penalty=l2)
-0.495239518171
-0.515471636075


In [8]:

print(test_score)
print(train_score)

-0.515471636075
-0.495239518171


In [9]:
prob=tpot.predict_proba(lda_test)

In [10]:
pro=pd.DataFrame(prob)

In [11]:
pro.to_csv('prob.csv')

In [12]:
# param={'n_estimators':range(201,271,10)}
# model = GridSearchCV(estimator=ExtraTreesClassifier(n_estimators=131,max_features=31,max_depth=5,min_samples_split=6,min_samples_leaf =2),param_grid=param,scoring='neg_log_loss',iid=False,cv=5)
# model.fit(lda_train,Y_train)
# model.best_params_, model.best_score_
ex=ExtraTreesClassifier(n_estimators=241,max_features=31,max_depth=5,min_samples_split=6,min_samples_leaf =2)
gpfeatures.training(ex,lda_train)

val_log_loss:  8.149693
train_log_loss 7.432956
Score:  0.784796573876
Val_score:  0.744142614601

test_score:  0.76404494382
log_loss:  0.493949


[('lda', 0.22646227234034877),
 ('gp_2', 0.11817169903620386),
 (24, 0.11061062324633372),
 ('gp_3', 0.0916128591960923),
 ('gp_5', 0.084466196571092902),
 ('gp_1', 0.084320090585762689),
 ('gp_4', 0.079876130353617028),
 (29, 0.034648778318090497),
 (2, 0.017435276691908815),
 (1, 0.016838050592347234),
 (22, 0.012158336705380364),
 (23, 0.011071507504068623),
 (0, 0.010223784796299052),
 (25, 0.0090635951970580685),
 (21, 0.0060941696121666281),
 (28, 0.0060662368974604827),
 (18, 0.0052505221442580401),
 (19, 0.0050003195904683708),
 (26, 0.004650356931670342),
 (27, 0.0046241488189948205),
 (14, 0.0045334739204302586),
 (17, 0.0045212329266378116),
 (30, 0.0044918775273849673),
 (10, 0.0044736251971315456),
 (16, 0.004450017109356939),
 (11, 0.0043260432537008545),
 (7, 0.0040562024892285657),
 (15, 0.0035963157458590452),
 (3, 0.0035531094677219684),
 (4, 0.0035248360258030588),
 (9, 0.0033681682651657864),
 (6, 0.0031883522313112993),
 (20, 0.0030717198393300292),
 (12, 0.0029101

In [13]:
# param={'min_samples_split':range(2,10)}
# model = GridSearchCV(estimator=RandomForestClassifier(n_estimators=170,max_features=32,max_depth=2,min_samples_split=52,param_grid=param,scoring='neg_log_loss',iid=False,cv=5)
# model.fit(lda_train,Y_train)
# model.best_params_, model.best_score_
rf=RandomForestClassifier(n_estimators=170,max_features=32,max_depth=2,min_samples_leaf=2)
gpfeatures.training(rf,lda_train)

val_log_loss:  7.567588
train_log_loss 8.431415
Score:  0.755888650964
Val_score:  0.740888066605

test_score:  0.780898876404
log_loss:  0.461713


[('lda', 0.72742107527921263),
 ('gp_3', 0.059363976249848967),
 ('gp_5', 0.057639884038842168),
 ('gp_4', 0.055328788600578903),
 ('gp_2', 0.051874003384802705),
 ('gp_1', 0.01436604431460619),
 (1, 0.0083940935778867524),
 (21, 0.0061224354629967354),
 (23, 0.0045509040196871206),
 (25, 0.0024112094068804478),
 (24, 0.0022858677256073832),
 (11, 0.0022512739100865775),
 (10, 0.0016687085358656614),
 (18, 0.0012173038038364849),
 (20, 0.0010874419636524373),
 (26, 0.00073570128989003934),
 (2, 0.00071944595268229902),
 (3, 0.0006281542578049277),
 (19, 0.00059568218109299134),
 (7, 0.00051455529296890032),
 (22, 0.00050119823555109502),
 (8, 0.0003222525156186762),
 (0, 0.0),
 (4, 0.0),
 (5, 0.0),
 (6, 0.0),
 (9, 0.0),
 (12, 0.0),
 (13, 0.0),
 (14, 0.0),
 (15, 0.0),
 (16, 0.0),
 (17, 0.0),
 (27, 0.0),
 (28, 0.0),
 (29, 0.0),
 (30, 0.0)]

In [14]:
# param_test1 = {'n_estimators':range(145,170)}
# gsearch1 = GridSearchCV(estimator = lgb.LGBMClassifier(boosting_type='dart',task='train',metric='binary_logloss',
#           objective = 'binary',
#           n_estimators=70,
#           silent = True,
#           min_data_in_leaf=1,
#           max_depth=4,
#           num_leaves=4,
#           learning_rate=0.04,
#           xgboost_dart_mode=True,
#           uniform_drop=False,
#           min_child_weight = 0.8,subsample=0.8,random_state=40), 
#                        param_grid = param_test1, scoring='neg_log_loss',iid=False,cv=5)
# gsearch1.fit(lda_train,Y_train)
# gsearch1.best_params_, gsearch1.best_score_

lgbm = lgb.LGBMClassifier(boosting_type='dart',task='train',metric='binary_logloss',
          objective = 'binary',
          n_estimators=161,
          silent = True,
          min_data_in_leaf=1,
          max_depth=2,
          num_leaves=4,
          learning_rate=0.04,
          xgboost_dart_mode=True,
          uniform_drop=False,
          min_child_weight = 0.8)
gpfeatures.training(lgbm,lda_train)

val_log_loss:  7.179507
train_log_loss 8.172564
Score:  0.763383297645
Val_score:  0.735615171138

test_score:  0.792134831461
log_loss:  0.460314


[('lda', 208),
 ('gp_5', 66),
 (23, 37),
 (3, 22),
 (1, 17),
 (17, 16),
 (19, 16),
 (21, 15),
 ('gp_4', 14),
 (26, 12),
 (15, 8),
 (18, 8),
 ('gp_1', 7),
 (14, 6),
 (22, 6),
 (5, 5),
 (24, 5),
 (7, 4),
 (25, 4),
 (2, 3),
 (9, 2),
 (12, 1),
 ('gp_3', 1),
 (0, 0),
 (4, 0),
 (6, 0),
 (8, 0),
 (10, 0),
 (11, 0),
 (13, 0),
 (16, 0),
 (20, 0),
 (27, 0),
 (28, 0),
 (29, 0),
 (30, 0),
 ('gp_2', 0)]

In [15]:


xgbo=xgb.XGBClassifier(n_estimators=38, learning_rate=0.06,max_depth=2,min_child_weight=0.15,
                                  max_features=2,)
gpfeatures.training(xgbo,lda_train)


# param_test1 = {'n_estimators':range(25,45)}
# gsearch1 = GridSearchCV(estimator = xgb.XGBClassifier(n_estimators=38, learning_rate=0.06,max_depth=5,min_child_weight=0.1,
#                                   max_features=2, subsample=0.8), 
#                        param_grid = param_test1, scoring='neg_log_loss',iid=False,cv=5)
# gsearch1.fit(lda_train,Y_train)
# gsearch1.best_params_, gsearch1.best_score_

val_log_loss:  7.955664
train_log_loss 8.283492
Score:  0.76017130621
Val_score:  0.739954725523

test_score:  0.769662921348
log_loss:  0.540043


[('lda', 0.64912283),
 (23, 0.070175439),
 (18, 0.03508772),
 (19, 0.03508772),
 ('gp_5', 0.03508772),
 (26, 0.02631579),
 ('gp_1', 0.02631579),
 (5, 0.01754386),
 (21, 0.01754386),
 (22, 0.01754386),
 ('gp_3', 0.01754386),
 (1, 0.0087719299),
 (3, 0.0087719299),
 (10, 0.0087719299),
 (15, 0.0087719299),
 (25, 0.0087719299),
 ('gp_4', 0.0087719299),
 (0, 0.0),
 (2, 0.0),
 (4, 0.0),
 (6, 0.0),
 (7, 0.0),
 (8, 0.0),
 (9, 0.0),
 (11, 0.0),
 (12, 0.0),
 (13, 0.0),
 (14, 0.0),
 (16, 0.0),
 (17, 0.0),
 (20, 0.0),
 (24, 0.0),
 (27, 0.0),
 (28, 0.0),
 (29, 0.0),
 (30, 0.0),
 ('gp_2', 0.0)]

In [19]:
# param_test1 = {'min_samples_split':range(2,10)}
# gsearch1 = GridSearchCV(estimator = GradientBoostingClassifier(learning_rate=0.002, min_samples_split=5, n_estimators=184,
#                                   min_samples_leaf=2,max_depth=14,max_features=57, subsample=0.8,random_state=10), 
#                        param_grid = param_test1, scoring='neg_log_loss',iid=False,cv=5)
# gsearch1.fit(gp_train,Y_train)
# gsearch1.best_params_, gsearch1.best_score_

# gb= GradientBoostingClassifier(n_estimators=184,learning_rate=0.002,max_depth=14,max_features=56,
#                                   min_samples_leaf=2,min_samples_split=5)
# gpfeatures.training(gb,lda_train)  



#可以不用

val_log_loss:  14.164986
train_log_loss 0.073960
Score:  0.997858672377
Val_score:  0.665938301485

test_score:  0.589887640449
log_loss:  0.678243


[('coaches_before_final', 0.33069701564575221),
 ('exp_win', 0.043608673858078352),
 ('stlrate_dis', 0.03377792649018755),
 ('oppfg2pct', 0.031796696871493779),
 ('tempo', 0.030395676555685744),
 ('opparate', 0.02694407012083299),
 ('f3grate', 0.025220654096838834),
 ('oppstlrate', 0.025063577442629343),
 ('f3_dis', 0.024145269765273522),
 ('oppf3grate', 0.023558872759209714),
 ('adjde', 0.021429884548086725),
 ('f3_score', 0.021390209269679675),
 ('ftpct', 0.016929063659486241),
 ('oppftpct', 0.016494431259381701),
 ('school_s16_ptg', 0.01614014392339734),
 ('career_school_wins', 0.016000363006440709),
 ('oppfg3pct', 0.015454947883580988),
 ('arate', 0.015248070027265708),
 ('overall_ncaa_ptg', 0.014969705770621627),
 ('team_season_wins', 0.014370322928917077),
 ('overall_s16_ptg', 0.014004245349304664),
 ('pct_score', 0.012468234659369564),
 ('de', 0.011666176635020972),
 ('pt_career_overall_losses', 0.01121817733333628),
 ('ex_coaches_before_final', 0.010608951660055023),
 ('fg2pct'

In [16]:
cat= cb.CatBoostClassifier(depth=5, learning_rate=0.03, loss_function='Logloss',leaf_estimation_method='Gradient',iterations=111,
                             bagging_temperature=2,od_type='Iter',od_wait=100,verbose=False,eval_metric='AUC')
gpfeatures.training(cat,lda_train)
# param_test1 = {'max_depth':range(2,8)}
# gsearch1 = GridSearchCV(estimator = cb.CatBoostClassifier(loss_function='Logloss',eval_metric='AUC',iterations=111,learning_rate=0.03,
#                                                           max_depth=6,verbose=False), 
#                        param_grid = param_test1, scoring='neg_log_loss',iid=False,cv=5)
# gsearch1.fit(lda_train,Y_train)
# gsearch1.best_params_, gsearch1.best_score_

val_log_loss:  7.373563
train_log_loss 7.137115
Score:  0.793361884368
Val_score:  0.732338137726

test_score:  0.786516853933
log_loss:  0.472806


[('lda', 27.137336691298465),
 ('gp_1', 5.2405370222964525),
 (21, 4.9655674927534141),
 (2, 4.2255286066411557),
 (18, 3.5868332002716339),
 (16, 3.2213373408032639),
 ('gp_4', 3.0146250298990482),
 ('gp_2', 2.9304153185113653),
 (23, 2.8569731227730832),
 (24, 2.8036528862414558),
 (0, 2.6739234398951335),
 (1, 2.6624444344442595),
 (20, 2.503581314802394),
 (7, 2.4929636077378596),
 (17, 2.2910209555067307),
 (10, 2.210786793253289),
 (19, 2.1822258926688596),
 (11, 2.1192323405487157),
 (15, 2.1154256211199707),
 (3, 1.9097724793866733),
 (25, 1.6182416578401759),
 ('gp_3', 1.4290663234054453),
 (28, 1.4224053197453674),
 ('gp_5', 1.4043572335017152),
 (22, 1.3465088998529389),
 (26, 1.3401409909281226),
 (14, 1.2982887266680609),
 (12, 1.2663195916721948),
 (13, 1.206394435067448),
 (4, 1.1750920611247222),
 (5, 0.80180562005744749),
 (9, 0.76103090110483018),
 (8, 0.73839242303750652),
 (6, 0.61561023205231025),
 (29, 0.2423857588911297),
 (30, 0.18977623419736089),
 (27, 0.0)]

In [17]:
# param_test1 = {'batch_size':range(70,90)}
# gsearch1 = GridSearchCV(estimator = MLPClassifier(hidden_layer_sizes=(18,),activation='logistic'), 
#                        param_grid = param_test1, scoring='neg_log_loss',iid=False,cv=5)
# gsearch1.fit(lda_train,Y_train)
# gsearch1.best_params_, gsearch1.best_score_
mlp=MLPClassifier(hidden_layer_sizes=(18,),activation='logistic',batch_size=80)
gpfeatures.training(mlp,lda_train)


val_log_loss:  8.731781
train_log_loss 8.357443
Score:  0.758029978587
Val_score:  0.732479861287

test_score:  0.747191011236
log_loss:  0.467077


In [133]:
# param_test1 = {'n_estimators':range(1,200,9)}
# gsearch1 = GridSearchCV(estimator =AdaBoostClassifier(LinearSVC(C=0.0006,intercept_scaling=0.002),algorithm='SAMME'), 
#                        param_grid = param_test1, scoring='accuracy',iid=False,cv=5)
# gsearch1.fit(lda_train,Y_train)
# gsearch1.best_params_, gsearch1.best_score_

# svc=LinearSVC(dual=True,C=0.0006,intercept_scaling=0.002)
# gpfeatures.training(svc,lda_train)

# svcb=BaggingClassifier(LinearSVC(dual=True,C=0.002,intercept_scaling=0.8),n_estimators=60)
# gpfeatures.training(svcb,lda_train)

# param_test1 = {'intercept_scaling':(0.002,0.005,0.008,0.01,0.015,0.02,0.025,0.03,0.04,0.08,0.0001,0.0002,0.0003,0.0005,0.0006,0.0007,0.0008,0.0009,0.001)}
# gsearch1 = GridSearchCV(estimator = LinearSVC(C=0.0006,intercept_scaling=0.002), 
#                        param_grid = param_test1, scoring='accuracy',iid=False,cv=5)
# gsearch1.fit(lda_train,Y_train)
# gsearch1.best_params_, gsearch1.best_score_

#没有 proba

val_log_loss:  9.119888
train_log_loss 8.727246
Score:  0.747323340471
Val_score:  0.742066731689

test_score:  0.73595505618


In [141]:

# sgd=BaggingClassifier(SGDClassifier(loss='log'),n_estimators=107)
# gpfeatures.training(sgd,lda_train)


# # param_test1 = {'n_estimators':range(100,150)}
# # gsearch1 = GridSearchCV(estimator = BaggingClassifier(SGDClassifier(loss='log')), 
# #                        param_grid = param_test1, scoring='neg_log_loss',iid=False,cv=10)
# # gsearch1.fit(lda_train,Y_train)
# # gsearch1.best_params_, gsearch1.best_score_

# # param_test1 = {'n_estimators':range(80,120)}
# # gsearch1 = GridSearchCV(estimator =AdaBoostClassifier(SGDClassifier(loss='log'),algorithm='SAMME'), 
# #                        param_grid = param_test1, scoring='accuracy',iid=False,cv=5)
# # gsearch1.fit(lda_train,Y_train)
# # gsearch1.best_params_, gsearch1.best_score_

# absgd = AdaBoostClassifier(SGDClassifier(loss='log'),
#                          algorithm="SAMME",
#                          n_estimators=100)
# gpfeatures.training(absgd,lda_train)

val_log_loss:  8.731821
train_log_loss 8.986102
Score:  0.73982869379
Val_score:  0.729192404484

test_score:  0.747191011236
log_loss:  0.517541
val_log_loss:  8.925864
train_log_loss 8.727244
Score:  0.747323340471
Val_score:  0.735736524325

test_score:  0.741573033708
log_loss:  0.588945


In [18]:
# param_test1 = {'n_estimators':range(1,100)}
# gsearch1 = GridSearchCV(estimator = BaggingClassifier(LogisticRegression(C=0.025)), 
#                        param_grid = param_test1, scoring='neg_log_loss',iid=False,cv=10)
# gsearch1.fit(lda_train,Y_train)
# gsearch1.best_params_, gsearch1.best_score_

# param_test1 = {'n_estimators':range(1,200)}   11
# gsearch1 = GridSearchCV(estimator =AdaBoostClassifier(LogisticRegression(C=0.025),algorithm='SAMME'), 
#                        param_grid = param_test1, scoring='accuracy',iid=False,cv=5)
# gsearch1.fit(lda_train,Y_train)
# gsearch1.best_params_, gsearch1.best_score_

# param_test1 = {'C':(0.026,0.024,0.027,0.023,0.015,0.02,0.025,0.03,0.04,0.08,0.012,0.013,0.014,0.016,0.017,0.018,0.019,0.0009,0.001)}
# gsearch1 = GridSearchCV(estimator = LogisticRegression(C=0.03), 
#                        param_grid = param_test1, scoring='neg_log_loss',iid=False,cv=5)
# gsearch1.fit(lda_train,Y_train)
# gsearch1.best_params_, gsearch1.best_score_

lr= BaggingClassifier(base_estimator=LogisticRegression(C=0.025), n_estimators=38)
gpfeatures.training(lr,lda_train)
# ablr = AdaBoostClassifier(LogisticRegression(C=0.025),
#                          algorithm="SAMME",
#                          n_estimators=100)
# gpfeatures.training(ablr,lda_train)

val_log_loss:  8.925828
train_log_loss 8.283488
Score:  0.76017130621
Val_score:  0.738789750629

test_score:  0.741573033708
log_loss:  0.520658


In [27]:
em=VotingClassifier([


#     ('mlp',mlp),
#      ('lr',lr),
      ('cat',cat),
     ('lgbm',lgbm),
#      ('xgbo',xgbo),
     ('rf',rf),
     ('ex',ex)


    
],voting='soft')
gpfeatures.training(em,lda_train)

val_log_loss:  9.702002
train_log_loss 7.395985
Score:  0.785867237687
Val_score:  0.749364387813

test_score:  0.719101123596
log_loss:  0.513702


In [28]:
from vecstack import StackingTransformer
es=[
     ('mlp',mlp),
      ('lr',lr),
      ('cat',cat),
     ('lgbm',lgbm),
      ('xgbo',xgbo),
     ('rf',rf),
     ('ex',ex),
     ('em',em)

]
stack=StackingTransformer(es,regression=True, verbose=2)
stack = stack.fit(lda_train, Y_train)
S_train = stack.transform(lda_train)
S_test=stack.transform(lda_test)


task:         [regression]
metric:       [mean_absolute_error]
variant:      [A]
n_estimators: [8]

estimator  0: [mlp: MLPClassifier]
    fold  0:  [0.29136691]
    fold  1:  [0.24100719]
    fold  2:  [0.28057554]
    fold  3:  [0.21942446]
    ----
    MEAN:     [0.25809353] + [0.02915389]

estimator  1: [lr: BaggingClassifier]
    fold  0:  [0.30215827]
    fold  1:  [0.24100719]
    fold  2:  [0.27697842]
    fold  3:  [0.22302158]
    ----
    MEAN:     [0.26079137] + [0.03078641]

estimator  2: [cat: CatBoostClassifier]
    fold  0:  [0.30215827]
    fold  1:  [0.24100719]
    fold  2:  [0.28776978]
    fold  3:  [0.21582734]
    ----
    MEAN:     [0.26169065] + [0.03481738]

estimator  3: [lgbm: LGBMClassifier]
    fold  0:  [0.28417266]
    fold  1:  [0.24460432]
    fold  2:  [0.28417266]
    fold  3:  [0.21582734]
    ----
    MEAN:     [0.25719424] + [0.02883313]

estimator  4: [xgbo: XGBClassifier]
    fold  0:  [0.29136691]
    fold  1:  [0.24820144]
    fold  2:  [0.284

In [30]:
S_train=pd.DataFrame(S_train)
S_test=pd.DataFrame(S_test)
f_train=pd.concat([lda_train,S_train],axis=1)
f_test=pd.concat([lda_test,S_test],axis=1)

In [29]:
training(lr,S_train)

val_log_loss:  9.896067
train_log_loss 8.690275
Score:  0.748394004283
Val_score:  0.744143216655

test_score:  0.713483146067
log_loss:  0.594418


In [39]:
X =f_train
y =Y_train
y = y.ravel()

# define random forest classifier, with utilising all cores and
# sampling in proportion to y labels
rf = ExtraTreesClassifier(n_jobs=-1, max_depth=8,n_estimators=247)

# define Boruta feature selection method
feat_selector = BorutaPy(rf, n_estimators='auto', verbose=2, max_iter=300,random_state=27)

# find all relevant features - 5 features should be selected
feat_selector.fit(X.values, y)

# check selected features - first 5 features are selected
feat_selector.support_

# check ranking of features
feat_selector.ranking_

# call transform() on X to filter it down to selected features
train = feat_selector.transform(X.values)
test=feat_selector.transform(f_test.values)
train=pd.DataFrame(train)
test=pd.DataFrame(test)

Iteration: 	1 / 300
Confirmed: 	0
Tentative: 	45
Rejected: 	0
Iteration: 	2 / 300
Confirmed: 	0
Tentative: 	45
Rejected: 	0
Iteration: 	3 / 300
Confirmed: 	0
Tentative: 	45
Rejected: 	0
Iteration: 	4 / 300
Confirmed: 	0
Tentative: 	45
Rejected: 	0
Iteration: 	5 / 300
Confirmed: 	0
Tentative: 	45
Rejected: 	0
Iteration: 	6 / 300
Confirmed: 	0
Tentative: 	45
Rejected: 	0
Iteration: 	7 / 300
Confirmed: 	0
Tentative: 	45
Rejected: 	0
Iteration: 	8 / 300
Confirmed: 	22
Tentative: 	10
Rejected: 	13
Iteration: 	9 / 300
Confirmed: 	22
Tentative: 	10
Rejected: 	13
Iteration: 	10 / 300
Confirmed: 	22
Tentative: 	10
Rejected: 	13
Iteration: 	11 / 300
Confirmed: 	22
Tentative: 	10
Rejected: 	13
Iteration: 	12 / 300
Confirmed: 	23
Tentative: 	6
Rejected: 	16
Iteration: 	13 / 300
Confirmed: 	23
Tentative: 	6
Rejected: 	16
Iteration: 	14 / 300
Confirmed: 	23
Tentative: 	6
Rejected: 	16
Iteration: 	15 / 300
Confirmed: 	23
Tentative: 	6
Rejected: 	16
Iteration: 	16 / 300
Confirmed: 	24
Tentative: 	5
Re

Rejected: 	20
Iteration: 	129 / 300
Confirmed: 	24
Tentative: 	1
Rejected: 	20
Iteration: 	130 / 300
Confirmed: 	24
Tentative: 	1
Rejected: 	20
Iteration: 	131 / 300
Confirmed: 	24
Tentative: 	1
Rejected: 	20
Iteration: 	132 / 300
Confirmed: 	24
Tentative: 	1
Rejected: 	20
Iteration: 	133 / 300
Confirmed: 	24
Tentative: 	1
Rejected: 	20
Iteration: 	134 / 300
Confirmed: 	24
Tentative: 	1
Rejected: 	20
Iteration: 	135 / 300
Confirmed: 	24
Tentative: 	1
Rejected: 	20
Iteration: 	136 / 300
Confirmed: 	24
Tentative: 	1
Rejected: 	20
Iteration: 	137 / 300
Confirmed: 	24
Tentative: 	1
Rejected: 	20
Iteration: 	138 / 300
Confirmed: 	24
Tentative: 	1
Rejected: 	20
Iteration: 	139 / 300
Confirmed: 	24
Tentative: 	1
Rejected: 	20
Iteration: 	140 / 300
Confirmed: 	24
Tentative: 	1
Rejected: 	20
Iteration: 	141 / 300
Confirmed: 	24
Tentative: 	1
Rejected: 	20
Iteration: 	142 / 300
Confirmed: 	24
Tentative: 	1
Rejected: 	20
Iteration: 	143 / 300
Confirmed: 	24
Tentative: 	1
Rejected: 	20
Iteration: 

In [26]:
# %store -r train
# %store -r test
# %store -r Y_train

x_train, x_test, y_train, y_test = train_test_split(train,
    Y_train, train_size=0.8, test_size=0.2)
tpot=TPOTClassifier(generations=10,population_size=20,scoring='neg_log_loss',verbosity=2,warm_start=True)
clf=tpot.fit(x_train, y_train)
train_score=tpot.score(x_train,y_train)
test_score=tpot.score(x_test, y_test)
tpot.export('tpot_pipeline.py')
print(train_score)
print(test_score)

Generation 1 - Current best internal CV score: -0.4916833782321247
Generation 2 - Current best internal CV score: -0.4916833782321247
Generation 3 - Current best internal CV score: -0.48595130314873963
Generation 4 - Current best internal CV score: -0.48595130314873963
Generation 5 - Current best internal CV score: -0.4853174723258755
Generation 6 - Current best internal CV score: -0.4853174723258755
Generation 7 - Current best internal CV score: -0.4853174723258755
Generation 8 - Current best internal CV score: -0.4825051793203664
Generation 9 - Current best internal CV score: -0.4825051793203664
Generation 10 - Current best internal CV score: -0.4825051793203664

Best pipeline: LogisticRegression(MaxAbsScaler(RFE(XGBClassifier(input_matrix, learning_rate=0.5, max_depth=4, min_child_weight=8, n_estimators=100, nthread=1, subsample=0.1), criterion=gini, max_features=0.45, n_estimators=100, step=0.75)), C=25.0, dual=False, penalty=l1)
-0.48025439195
-0.564363082581


In [27]:
print(train_score)
print(test_score)

-0.48025439195
-0.564363082581


In [33]:
%store -r test
prob=clf.predict_proba(test)

In [34]:
testprob=pd.DataFrame(prob)

In [35]:
testprob.to_csv('probability.csv')

In [8]:
print(train_score)
print(test_score)

-0.473708985515
-0.504151913254


In [33]:
x_train, x_test, y_train, y_test = train_test_split(f_train,
    Y_train, train_size=0.8, test_size=0.2)
tpot=TPOTClassifier(generations=10,population_size=20,scoring='neg_log_loss',verbosity=2,warm_start=True)
clf2=tpot.fit(x_train, y_train)
train_score=tpot.score(x_train,y_train)
test_score=tpot.score(x_test, y_test)
tpot.export('tpot_pipeline.py')
print(train_score)
print(test_score)

(2278, 45)

In [40]:
test.shape

(2278, 24)

In [38]:
%store f_train
%store f_test

Stored 'f_train' (DataFrame)
Stored 'f_test' (DataFrame)


In [45]:
%store train
%store test

Stored 'train' (DataFrame)
Stored 'test' (DataFrame)


In [29]:
train.columns

RangeIndex(start=0, stop=24, step=1)

In [30]:
test.columns

RangeIndex(start=0, stop=2, step=1)

In [31]:
%store -r test

In [32]:
test.columns

RangeIndex(start=0, stop=24, step=1)

In [85]:
import numpy as np
import pandas as pd
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline, make_union
from sklearn.preprocessing import MaxAbsScaler
from tpot.builtins import StackingEstimator
from xgboost import XGBClassifier
%store -r train
pipe=make_pipeline(
    StackingEstimator(estimator=XGBClassifier(learning_rate=0.5, max_depth=4, min_child_weight=8, n_estimators=100, nthread=1, subsample=0.1)),
    RFE(estimator=ExtraTreesClassifier(criterion="gini", max_features=0.45, n_estimators=100), step=0.7500000000000001),
    MaxAbsScaler(),
    LogisticRegression(C=25.0, dual=False, penalty="l1")
)

x_train, x_test, y_train, y_test = train_test_split(train,
    Y_train, train_size=0.8, test_size=0.2)
mean_impute_scores = cross_val_score(pipe, x_train.values, y_train,
                                         scoring='neg_log_loss',
                                         cv=10)


In [86]:
np.mean(mean_impute_scores)

-0.4903638405079162